```
This notebook sets up and runs a set of benchmarks to compare
different numerical discretizations of the SWEs

Copyright (C) 2016  SINTEF ICT

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Import packages we need
import numpy as np
from matplotlib import animation, rc
from matplotlib import pyplot as plt

import os
import pyopencl
import datetime
import sys

# requires netcdf4-python (netcdf4-python.googlecode.com)
from netCDF4 import Dataset as NetCDFFile

#Finally, import our simulator
from SWESimulators import FBL, CTCS, DataOutput

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

#Finally, import our simulator
from SWESimulators import FBL, CTCS, LxF, FORCE, HLL, HLL2, KP07, KP07_dimsplit, WAF, CDKLM16, DataOutput, PlotHelper, Common

In [ ]:
#Make sure we get compiler output from OpenCL
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"

#Set which CL device to use, and disable kernel caching
if (str.lower(sys.platform).startswith("linux")):
    os.environ["PYOPENCL_CTX"] = "0"
else:
    os.environ["PYOPENCL_CTX"] = "1"
os.environ["CUDA_CACHE_DISABLE"] = "1"
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"
os.environ["PYOPENCL_NO_CACHE"] = "1"

#Create OpenCL context
cl_ctx = pyopencl.create_some_context()
print "Using ", cl_ctx.devices[0].name

In [ ]:
#Create output directory for images
imgdir='images_' + datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
os.makedirs(imgdir)
print "Saving images to " + imgdir

In [ ]:
## A common initial condition maker:
def makeCornerBump(eta, nx, ny, dx, dy, halo):
    x_center = 4*dx
    y_center = 4*dy
    for j in range(-halo[2], ny + halo[0]):
        for i in range(-halo[3], nx + halo[1]):
            x = dx*i - x_center
            y = dy*j - y_center
            size = 500.0*min(dx, dy)
            if (np.sqrt(x**2 + y**2) < size):
                eta[j+halo[2], i+halo[3]] = np.exp(-(x**2/size+y**2/size))
                
def makeUpperCornerBump(eta, nx, ny, dx, dy, halo):
    x_center = (nx-4)*dx
    y_center = (ny-4)*dy
    for j in range(-halo[2], ny + halo[0]):
        for i in range(-halo[3], nx + halo[1]):
            x = dx*i - x_center
            y = dy*j - y_center
            size = 500.0*min(dx, dy)
            if (np.sqrt(x**2 + y**2) < size):
                eta[j+halo[2], i+halo[3]] = np.exp(-(x**2/size+y**2/size))

                
def makeCentralBump(eta, nx, ny, dx, dy, halo):
    x_center = dx*nx/2.0
    y_center = dy*ny/2.0
    for j in range(-halo[2], ny + halo[0]):
        for i in range(-halo[3], nx + halo[1]):
            x = dx*i - x_center
            y = dy*j - y_center
            size = (0.015* min(nx, ny)*min(dx, dy))**2
            if (np.sqrt(x**2 + y**2) < size):
                eta[j+halo[2], i+halo[3]] = np.exp(-(x**2/size+y**2/size))
                
def makeLowerLeftBump(eta, nx, ny, dx, dy, halo):
    x_center = dx*nx*0.3
    y_center = dy*ny*0.2
    for j in range(-halo[2], ny + halo[0]):
        for i in range(-halo[3], nx + halo[1]):
            x = dx*i - x_center
            y = dy*j - y_center
            size = 500.0*min(dx, dy)
            if (np.sqrt(x**2 + y**2) < size):
                eta[j+halo[2], i+halo[3]] = np.exp(-(x**2/size+y**2/size))
                
                
## Adding initial conditions on top of an existing initialCondition:
def addCornerBump(eta, nx, ny, dx, dy, halo):
    x_center = 4*dx
    y_center = 4*dy
    for j in range(-halo[2], ny + halo[0]):
        for i in range(-halo[3], nx + halo[1]):
            x = dx*i - x_center
            y = dy*j - y_center
            size = 500.0*min(dx, dy)
            if (np.sqrt(x**2 + y**2) < size):
                eta[j+halo[2], i+halo[3]] += np.exp(-(x**2/size+y**2/size))
                
                
def addUpperCornerBump(eta, nx, ny, dx, dy, halo):
    x_center = (nx-4)*dx
    y_center = (ny-4)*dy
    for j in range(-halo[2], ny + halo[0]):
        for i in range(-halo[3], nx + halo[1]):
            x = dx*i - x_center
            y = dy*j - y_center
            size = 500.0*min(dx, dy)
            if (np.sqrt(x**2 + y**2) < size):
                eta[j+halo[2], i+halo[3]] += np.exp(-(x**2/size+y**2/size))

                
def addCentralBump(eta, nx, ny, dx, dy, halo):
    x_center = dx*nx/2.0
    y_center = dy*ny/2.0
    for j in range(-halo[2], ny + halo[0]):
        for i in range(-halo[3], nx + halo[1]):
            x = dx*i - x_center
            y = dy*j - y_center
            size = 500.0*min(dx, dy)
            #size = (0.015* min(nx, ny)*min(dx, dy))**2
            if (np.sqrt(x**2 + y**2) < size):
                eta[j+halo[2], i+halo[3]] += np.exp(-(x**2/size+y**2/size))
                
                
                
def addLowerLeftBump(eta, nx, ny, dx, dy, halo):
    print("addLowerLeftBump")
    print("halo", halo)
    print("nx", nx)
    print("ny", ny)
    x_center = dx*nx*0.3
    y_center = dy*ny*0.2
    for j in range(-halo[2], ny + halo[0]):
        for i in range(-halo[3], nx + halo[1]):
            x = dx*i - x_center
            y = dy*j - y_center
            size = 500.0*min(dx, dy)
            if (np.sqrt(x**2 + y**2) < size):
                eta[j+halo[2], i+halo[3]] += np.exp(-(x**2/size+y**2/size))
            
# This bump is for debug purposes and will be modified without mercy :)
def addDebugBump(eta, nx, ny, dx, dy, posx, posy, halo):
    x_center = dx*nx*posx
    y_center = dy*ny*posy
    for j in range(-halo[2], ny + halo[0]):
        for i in range(-halo[3], nx + halo[1]):
            x = dx*i - x_center
            y = dy*j - y_center
            size = 500.0*min(dx, dy)
            if (np.sqrt(x**2 + y**2) < size):
                eta[j+halo[2], i+halo[3]] += np.exp(-(x**2/size+y**2/size))
                
def addTopographyBump(h, nx, ny, dx, dy, halo, bumpsize):
    # Creating a bump in y direction (uniform in x direction)
    yPos = np.floor(ny*0.25)
    print(yPos)
    print((-halo[2], ny))
    print((-halo[3], nx + halo[1]))
    for j in range(-halo[2], ny + halo[0]):
        for i in range(-halo[3], nx + halo[1]):
            r = 0.01*(yPos - j)**2
            h[j+halo[2], i+halo[3]] -= bumpsize*np.exp(-r) 

def makeBathymetryCrater(B, nx, ny, dx, dy, halo):
    x_center = dx*nx/2.0
    y_center = dy*ny/2.0
    minReach = min(nx*dx, ny*dy)
    innerEdge = minReach*0.3/2.0
    outerEdge = minReach*0.7/2.0
    for j in range(-halo[2], ny + halo[0]):
        for i in range(-halo[3], nx + halo[1]):
            x = dx*i - x_center
            y = dy*j - y_center
            radius = np.sqrt(x**2 + y**2)
            if (radius > innerEdge) and (radius < outerEdge):
                B[j+halo[2], i+halo[3]] = 30.0*np.sin((radius - innerEdge)/(outerEdge - innerEdge)*np.pi )**2
            else:
                B[j+halo[2], i+halo[3]] = 0.0
                
def makeBathymetryCrazyness(B, nx, ny, dx, dy, halo):
    length = dx*nx*1.0
    height = dy*ny*1.0
    for j in range(-halo[2], ny + halo[0]):
        for i in range(-halo[3], nx + halo[1]):
            x = dx*i*1.0
            y = dy*j*1.0
            B[j+halo[2], i+halo[3]] = 25.0*(np.sin(np.pi*(x/length)*4)**2 + np.sin(np.pi*(y/height)*4)**2)
            
def linearBathymetryX(B, nx, ny, dx, dy, halo, low, high):
    length=dx*nx*1.0
    gradient = (high-low)/length
    for j in range(0, ny+1):
        for i in range(0, nx+1):
            B[j+halo[2], i+halo[3]] = low + i*dx*gradient
            
def linearBathymetryY(B, nx, ny, dx, dy, halo, low, high):
    length=dy*ny*1.0
    gradient = (high-low)/length
    for j in range(0, ny+1):
        for i in range(0, nx+1):
            B[j+halo[2], i+halo[3]] = low + j*dy*gradient
            
            
def diagonalWallBathymetry(B, nx, ny, dx, dy, halo, height):
    for j in range(0, ny+1):
        for i in range(0, nx+1):
            factor = 1.0
            if ( i-j > -30 and i-j < 10):
                factor = 1 - np.exp(-0.01*(abs(10 - j + i)**2))
            B[j+halo[2], i+halo[3]] = factor*height*np.exp(-0.006*(abs(100-j - i)**2))

                
def addDiagonalDam(h, nx, ny, dx, dy, halo, height):
    for j in range(0, ny+1):
        for i in range(0, nx+1):
            if ( i+j < 50):
                h[j+halo[2], i+halo[3]] += height
                
def straightWallBathymetry(B, nx, ny, dx, dy, halo, height):
    for j in range(0, ny+1):
        for i in range(0, nx+1):
            factor = 1.0
            if ( i > 40 and i < 60):
                factor = 1 - np.exp(-0.05*(abs(50 - i)**2))
            B[j+halo[2], i+halo[3]] = factor*height*np.exp(-0.01*(abs(80-j)**2))

def addStraightDam(h, nx, ny, dx, dy, halo, height):
    for j in range(0, ny+1):
        for i in range(0, nx+1):
            if ( j < 30):
                h[j+halo[2], i+halo[3]] += height

In [ ]:
ghosts = [3,3,3,3]

B = np.zeros((ny+ghosts[0]+ghosts[2], nx+ghosts[1]+ghosts[3]), dtype=np.float32);
#makeBathymetryCrazyness(h0, nx, ny, dx, dy, ghosts)
#linearBathymetryY(B, nx, ny, dx, dy, ghosts, 5, 30)
diagonalWallBathymetry(B, nx, ny, dx, dy, ghosts, 50)
reload(PlotHelper)
fig = plt.figure()
singlePlotter = PlotHelper.SinglePlot(fig, x_coords, y_coords, B, interpolation_type="None")

print(h0.shape)
h0shapeX, h0shapeY = h0.shape
print(np.min(h0), np.max(h0))
print(range(0,2))
fig = plt.figure()
plt.plot(B[82, :])


In [ ]:
# Forward backward linear
nx = 100
ny = 200

dx = 200.0
dy = 200.0

dt = 1
g = 9.81
f = 0.0
r = 0.0


bcSettings = 1
if (bcSettings == 1):
    boundaryConditions = Common.BoundaryConditions()
    ghosts = [0,0,0,0] # north, east, south, west
elif (bcSettings == 2):
    boundaryConditions = Common.BoundaryConditions(2,2,2,2)
    ghosts = [1,1,0,0] # Both periodic
elif bcSettings == 3:
    boundaryConditions = Common.BoundaryConditions(2,1,2,1)
    ghosts = [1,0,0,0] # periodic north-south
else:
    boundaryConditions = Common.BoundaryConditions(1,2,1,2)
    ghosts = [0,1,0,0] # periodic east-west


h0 = np.ones((ny+ghosts[0], nx+ghosts[1]), dtype=np.float32) * 60;
addTopographyBump(h0, nx, ny, dx, dy, ghosts, 40)

eta0 = np.zeros((ny+ghosts[0], nx+ghosts[1]), dtype=np.float32);
u0 = np.zeros((ny+ghosts[0], nx+1), dtype=np.float32);
v0 = np.zeros((ny+1, nx+ghosts[1]), dtype=np.float32);

#Create bump in to lower left of domain for testing
x_center = dx*nx*0.3
y_center = dy*ny*0.2
#makeLowerLeftBump(eta0, nx, ny, dx, dy, ghosts)
#makeCornerBump(eta0, nx, ny, dx, dy, ghosts)
makeCentralBump(eta0, nx, ny, dx, dy, ghosts)


#Initialize simulator
reload(FBL)

#wind = Common.WindStressParams(type=0, tau0=0.1, rho=1025, alpha=0.000005)
#wind = Common.WindStressParams(type=0, tau0=3.0, rho=1025, alpha=1.0/(100*dx))
wind = Common.WindStressParams(type=99)
f = 0.0

sim = FBL.FBL(cl_ctx, \
              h0, eta0, u0, v0, \
              nx, ny, \
              dx, dy, dt, \
              g, f, r, \
              wind_stress=wind, \
              boundary_conditions=boundaryConditions)


#Calculate radius from center of bump for plotting
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))
print("radius.shape", radius.shape)

if (ghosts[0] == 0):
    ghosts[0] = None
else:
    ghosts[0] = -ghosts[0]
if (ghosts[1] == 0):
    ghosts[1] = None
else:
    ghosts[1] = -ghosts[1]

    
fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, \
                                eta0[ghosts[2]:ghosts[0], ghosts[3]:ghosts[1]],\
                                u0[ghosts[2]:ghosts[0], :], \
                                v0[:, ghosts[3]:ghosts[1]])

def animate(i):
    if (i>0):
        #t = sim.step(1.0)
        t = sim.step(10.0)
    else:
        t = 0.0
    eta1, u1, v1 = sim.download()

    plotter.plot(eta1[ghosts[2]:ghosts[0], ghosts[3]:ghosts[1]],\
                 u1[ghosts[2]:ghosts[0], :], \
                 v1[:, ghosts[3]:ghosts[1]]);

    fig.suptitle("FBL Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%10 == 0):
        print "{:03.0f}".format(100*i / 20.0) + " % => t=" + str(t)
        fig.savefig(imgdir + "/{:010.0f}_fbl.png".format(t))
        #print(np.linalg.norm(eta1[:, 0] - eta1[:, nx-1]))
        #print(np.linalg.norm(eta1[0, :] - eta1[ny-1, :]))
            
anim = animation.FuncAnimation(fig, animate, range(100), interval=100)
plt.close(anim._fig)
anim

In [ ]:
print(sim)
print("Wind type", wind.type)
print("boundaryConditions", bcSettings)
print("Coriolis", f)

eta1, u1, v1 = sim.download()


fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius,
                 eta1[ghosts[2]:ghosts[0], ghosts[3]:ghosts[1]],\
                 u1[ghosts[2]:ghosts[0], :], \
                 v1[:, ghosts[3]:ghosts[1]]);


In [ ]:
print(imgdir)

In [ ]:
#Centered in time, centered in space

nx = 100
ny = 200

dx = 200.0
dy = 200.0

dt = 1
g = 9.81
f = 0.0
r = 0.0
A = 1


bcSettings = 1
ghosts = [1,1,1,1]
if (bcSettings == 1):
    boundaryConditions = Common.BoundaryConditions()
    #ghosts = [1,1,1,1] # north, east, south, west
elif (bcSettings == 2):(Bi, nx+1, ny+1, dx, dy, ghosts)
    boundaryConditions = Common.BoundaryConditions(2,2,2,2)
    #ghosts = [2,2,2,2] # Both periodicself.cl_data.swap()self.cl_data.swap()
elif bcSettings == 3:
    boundaryConditions = Common.BoundaryConditions(2,1,2,1)
    #ghosts = [2,1,1,1] # periodic north-south
else:
    boundaryConditions = Common.BoundaryConditions(1,2,1,2)
    #ghosts = [1,2,1,1] # periodic east-west

h0 = np.ones((ny+2, nx+2), dtype=np.float32, order='C') * 60;
addTopographyBump(h0, nx, ny, dx, dy, ghosts, 40)

eta0 = np.zeros((ny+2, nx+2), dtype=np.float32, order='C');
u0 = np.zeros((ny+2, nx+1+2), dtype=np.float32, order='C');
v0 = np.zeros((ny+1+2, nx+2), dtype=np.float32, order='C');

#Create bump in to lower left of domain for testing
x_center = dx*nx*0.3
y_center = dy*ny*0.2
#makeUpperCornerBump(eta0, nx, ny, dx, dy, ghosts)
makeCentralBump(eta0, nx, ny, dx, dy, ghosts)
#makeCornerBump(eta0, nx, ny, dx, dy, ghosts)
#makeLowerLeftBump(eta0, nx, ny, dx, dy, ghosts)
 
    
#wind = WindStressParams(type=0, tau0=0.1, rho=1025, alpha=0.000005)
#wind = Common.WindStressParams(type=0, tau0=3.0, rho=1025, alpha=1.0/(100*dx))
wind = Common.WindStressParams(type=99)
#f = 0.01
    
#Initialize simulator
reload(CTCS)
sim = CTCS.CTCS(cl_ctx, \
                h0, eta0, u0, v0, \
                nx, ny, \
                dx, dy, dt, \
                g, f, r, A, \
                wind_stress=wind, \
                boundary_conditions=boundaryConditions )


#Calculate radius from center of bump for plotting
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_coords = np.subtract(x_coords, x_center)(Bi, nx+1, ny+1, dx, dy, ghosts)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))

ghosts = [-1,1,-1,1]

fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, eta0[1:-1, 1:-1], u0[1:-1, :], v0[:, 1:-1])


def animate(i):
    if (i>0):
        t = sim.step(10.0)
    else:
        t = 0.0
    eta1, u1, v1 = sim.download()

    plotter.plot(eta1[1:-1, 1:-1], u1[1:-1, :], v1[:, 1:-1]);
    fig.suptitle("CTCS Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%10 == 0):
        print "{:03.0f}".format(100*i / 50.0) + " % => t=" + str(t)
        fig.savefig(imgdir + "/{:010.0f}_ctcs.png".format(t))

anim = animation.FuncAnimation(fig, animate, range(100), interval=100)
plt.close(anim._fig)
anim

In [ ]:

#Coriolis well balanced reconstruction scheme
nx = 100
ny = 200

dx = 200.0
dy = 200.0

dt = 0.95/10.0
g = 9.81

f = 0.00
r = 0.0


bcSettings = 1
ghosts = np.array([3,3,3,3]) # north, east, south, west
validDomain = np.array([3,3,3,3])
if (bcSettings == 1):
    boundaryConditions = Common.BoundaryConditions()
elif (bcSettings == 2):
    boundaryConditions = Common.BoundaryConditions(2,2,2,2)
elif bcSettings == 3:
    # Periodic NS
    boundaryConditions = Common.BoundaryConditions(2,1,2,1)
else:
    # Periodic EW
    boundaryConditions = Common.BoundaryConditions(1,2,1,2)
    
dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])
#print(dataShape)


waterHeight = 60
h0 = np.ones(dataShape, dtype=np.float32, order='C') * waterHeight;
u0 = np.zeros(dataShape, dtype=np.float32, order='C');
v0 = np.zeros(dataShape, dtype=np.float32, order='C');
#print(v0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]].shape)

# Bathymetry:
Bi = np.zeros((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C')
#diagonalWallBathymetry(Bi, nx, ny, dx, dy, ghosts, 50)
#straightWallBathymetry(Bi, nx, ny, dx, dy, ghosts, 50)
makeBathymetryCrater(Bi, nx+1, ny+1, dx, dy, ghosts)
#makeBathymetryCrazyness(Bi, nx+1, ny+1, dx, dy, ghosts)
#linearBathymetryX(Bi, nx, ny, dx, dy, ghosts, 5, 50)
#linearBathymetryY(Bi, nx, ny, dx, dy, ghosts, 5, 50)

#addCornerBump(h0, nx, ny, dx, dy, validDomain)
#addUpperCornerBump(h0, nx, ny, dx, dy, validDomain)
#addCentralBump(h0, nx, ny, dx, dy, validDomain)
#addLowerLeftBump(h0, nx, ny, dx, dy, validDomain)
#addDebugBump(h0, nx, ny, dx, dy, 0.2, 0.1, validDomain)
#addStraightDam(h0, nx, ny, dx, dy, ghosts, 0.1)


# WIND
#wind = Common.WindStressParams(type=0, tau0=3.0, rho=1025, alpha=1.0/(100*dx))
wind = Common.WindStressParams(type=99)
f = 0.0

#Initialize simulator
reload(CDKLM16)
reload(Common)
sim = CDKLM16.CDKLM16(cl_ctx, \
                h0, u0, v0, \
                Bi, \
                nx, ny, \
                dx, dy, dt, \
                g, f, r, \
                wind_stress=wind, \
                boundary_conditions=boundaryConditions)


#Calculate radius from center of bump for plotting
x_center = dx*nx/2.0
y_center = dy*ny/2.0
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))


fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, 
                                h0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]] - waterHeight, 
                                u0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                                v0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]])

def animate(i):
    if (i>0):
        t = sim.step(10.0)
    else:
        t = 0.0
    h1, u1, v1 = sim.download()
    
    brighten = 1
    
    plotter.plot(h1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]] - waterHeight, 
                 brighten*u1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                 brighten*v1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]]);
    fig.suptitle("CDKLM16 Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%10 == 0):
        print "{:03.0f}".format(100*i / 50.0) + " % => t=" + str(t) + "\tMax h: " + str(np.max(h1))
        fig.savefig(imgdir + "/{:010.0f}_cdklm16.png".format(t))
             
anim = animation.FuncAnimation(fig, animate, range(100), interval=100)
plt.close(anim._fig)
anim

In [ ]:
h3, hu3, hv3 = sim.download()
print("Initial water volume: ", sum(sum(h0)))
print("After simulation:     ", sum(sum(h3)))
print(imgdir)
print("min-max h:            ", np.min(h3), np.max(h3))
print("min-max hu:           ", np.min(hu3), np.max(hu3))
print("min-max hv:           ", np.min(hv3), np.max(hv3))


plt.plot(hu3[:,50])

In [ ]:
h1, u1, v1 = sim.download()
fig = plt.figure()
singlePlotter = PlotHelper.SinglePlot(fig, x_coords, y_coords, h1)
fig = plt.figure()
singlePlotter = PlotHelper.SinglePlot(fig, x_coords, y_coords, v1)


In [ ]:
h1, u1, v1 = sim.download()
fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius,
                 h1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]] - waterHeight, 
                 u1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                 v1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]]);

print(sum(sum(h0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]])))
print(sum(sum(h1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]])))
print(60*nx*ny)
print(7./20000/500)
print(imgdir)

print("---- Finding largest eigenvalue ---- ")
a = np.array([[1,2], [3,4]])
b = np.array([[2,1], [4,3]])
c = np.abs(np.array([[2,10], [-40,3]]))
print(np.max(np.maximum(a,np.maximum(b,c))))

h = h1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]]
u = u1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]] 
v = v1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]]

lum = np.max(np.abs(u - np.sqrt(g*h)))
lup = np.max(np.abs(u + np.sqrt(g*h)))
lvm = np.max(np.abs(v - np.sqrt(g*h)))
lvp = np.max(np.abs(v + np.sqrt(g*h)))

eigmax = max( lum, lup, lvm, lvp)

0.25*dx/eigmax

In [ ]:
h1, u1, v1 = sim.download()
#print(e)
print(max(h1[:,4]))
print(max(h1[4,:]))
print("h1.shape", h1.shape)
print("(ny, nx)", (ny, nx))
print("h1[4,:].shape", h1[4,:].shape)

i = 3
print("np.linalg.norm(h1[:,i] - h1[:,nx+i])", np.linalg.norm(h1[:,i] - h1[:,nx+i]))
plt.plot(h1[:,i], 'bd')
plt.plot(h1[:, i+nx], 'r')
plt.plot(h1[i,:], 'gd')
plt.plot(h1[i+ny, :], 'k')
waterHeight
#print(np.linalg.norm(h1[,:] - h1[ny+10, :]))

In [ ]:
# Kurganov-Petrova 2007 paper
reload(KP07)
nx = 100
ny = 200

dx = 200.0
dy = 200.0

dt = 0.95/100
g = 9.81

f = 0.00
r = 0.0

bcSettings = 1
ghosts = np.array([2,2,2,2]) # north, east, south, west
validDomain = np.array([2,2,2,2])
if (bcSettings == 1):
    boundaryConditions = Common.BoundaryConditions()
elif (bcSettings == 2):
    boundaryConditions = Common.BoundaryConditions(2,2,2,2)
elif bcSettings == 3:
    # Periodic NS
    boundaryConditions = Common.BoundaryConditions(2,1,2,1)
else:
    # Periodic EW
    boundaryConditions = Common.BoundaryConditions(1,2,1,2)
    
dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])
waterHeight = 60
h0 = np.ones(dataShape, dtype=np.float32, order='C') * waterHeight;
u0 = np.zeros(dataShape, dtype=np.float32, order='C');
v0 = np.zeros(dataShape, dtype=np.float32, order='C');

Bi = np.zeros((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C')
#makeBathymetryCrater(Bi, nx+1, ny+1, dx, dy, ghosts)
#makeBathymetryCrazyness(Bi, nx+1, ny+1, dx, dy, ghosts)
#linearBathymetryX(Bi, nx, ny, dx, dy, ghosts, 5, 50)
#diagonalWallBathymetry(Bi, nx, ny, dx, dy, ghosts, 50)
#straightWallBathymetry(Bi, nx, ny, dx, dy, ghosts, 50)

#addCornerBump(h0, nx, ny, dx, dy, validDomain)
#addUpperCornerBump(h0, nx, ny, dx, dy, validDomain)
addCentralBump(h0, nx, ny, dx, dy, validDomain)
#addLowerLeftBump(h0, nx, ny, dx, dy, validDomain)
#addDebugBump(h0, nx, ny, dx, dy, 0.2, 0.1, validDomain)
#addDiagonalDam(h0, nx, ny, dx, dy, ghosts, 0.05)
#addStraightDam(h0, nx, ny, dx, dy, ghosts, 0.1)


# WIND
#wind = Common.WindStressParams(type=0, tau0=3.0, rho=1025, alpha=1.0/(100*dx))
wind = Common.WindStressParams(type=99)
f = 0.00

#Initialize simulator
reload(KP07)
reload(Common)
sim = KP07.KP07(cl_ctx, \
                h0, Bi, u0, v0, \
                nx, ny, \
                dx, dy, dt, \
                g, f, r, \
                wind_stress=wind, \
                boundary_conditions=boundaryConditions,
                use_rk2=True)


#Calculate radius from center of bump for plotting
x_center = dx*nx/2.0
y_center = dy*ny/2.0
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))


fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, 
                                h0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]] - waterHeight, 
                                u0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                                v0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]] )


def animate(i):
    if (i>0):
        t = sim.step(10.0)
    else:
        t = 0.0
    h1, u1, v1 = sim.download()

    plotter.plot(h1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]] - waterHeight, 
                 u1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                 v1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]] );
    fig.suptitle("KP07 Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%10 == 0):
        print "{:03.0f}".format(100*i / 50.0) + " % => t=" + str(t) + "\tMax h: " + str(np.max(h1))
        #fig.savefig(imgdir + "/{:010.0f}_kp07.png".format(t))
             
anim = animation.FuncAnimation(fig, animate, range(100), interval=100)
plt.close(anim._fig)
anim

In [ ]:
h3, hu3, hv3 = sim.download()
print("Initial water volume: ", sum(sum(h0)))
print("After simulation:     ", sum(sum(h3)))
print(imgdir)
print("min-max h:           ", np.min(h3), np.max(h3))
print("min-max hu:           ", np.min(hu3), np.max(hu3))
print("min-max hv:           ", np.min(hv3), np.max(hv3))

In [ ]:
# Quickly testing initBm
Bi2, Bm2 = sim.downloadBathymetry()

y_coords, x_coords = np.mgrid[0:(ny+1)*dy:dy, 0:(nx+1)*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)

fig = plt.figure()
singlePlotter = PlotHelper.SinglePlot(fig, x_coords, y_coords, Bi2)

y_coords, x_coords = np.mgrid[0:(ny)*dy:dy, 0:(nx)*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)

fig = plt.figure()
singlePlotter = PlotHelper.SinglePlot(fig, x_coords, y_coords, Bm2)

print(np.max(Bi2))
print(np.max(Bm2))

In [ ]:
plt.plot(Bm2[ 50, :])
print(Bm2.shape)
print(np.max(Bm2), np.min(Bm2))
print(Bm2[50, :])

In [ ]:
a = [1,2,3,4]
a[1]
eta1, hu1, hv1 = sim.download()
print(eta1.max())
print(eta1.min())
print(hu1.max())
print(hu1.min())
print(hv1.max())
print(hv1.min())
#eta0 = np.zeros((5,4), dtype=np.float32);
#print(eta0)
#print(eta0.shape)
#print(eta0[0:None,
#           1:-0])
print(ghosts)
if (ghosts[0] == None):
    ghosts[0] = 0

for i in range(0, nx+ghosts[1]-1) :
    for j in range(0, ny+ghosts[0] -1):
        if eta1[j,i] != eta1[j,i]:
            print("i: " + str(i) + " and j: " + str(j))




#eta0 = np.zeros((ny+ghosts[0], nx+ghosts[1])

print(nx)
print(dx)
x = np.linspace(dx/2,(nx-0.5)*dx, nx)
print(x.shape)
#print(x)
print(np.linspace(0,9,10))

B = lambda: None
setattr(B, 'h', "hei") 
hasattr(B, 'h')
B.h

ghosts*2-5